In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv('train.csv')
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [14]:
data = df.copy()
data.drop(['datetime','atemp'],axis = 1,inplace=True)

In [15]:
## there is the chance of data leakage so now itself we need to split the data into train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data,data['count'],test_size=0.2,random_state=100)

In [16]:
X_train.shape , X_test.shape

((8708, 10), (2178, 10))

In [17]:
##now from EDA we see that some of the continous features are skewed
##so, inorder to balance it we use log normal distrubution
applied_to = ['temp','count']
for feature in applied_to:
    data[feature]=np.log(data[feature])

In [18]:
data.head()

,season,holiday,workingday,weather,temp,humidity,windspeed,casual,registered,count
0,1,0,0,1,2.286456,81,0.0,3,13,2.772589
1,1,0,0,1,2.199444,80,0.0,8,32,3.688879
2,1,0,0,1,2.199444,80,0.0,5,27,3.465736
3,1,0,0,1,2.286456,75,0.0,3,10,2.564949
4,1,0,0,1,2.286456,75,0.0,0,1,0.000000


In [19]:
##since we have the ranges in data ,perfroming minmax scaler and storing in seperate variable
scaling_feature = [feature for feature in data.columns if feature not in ['count']]
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(data[scaling_feature])


MinMaxScaler()

In [20]:
scaler.transform(data[scaling_feature])

array([[0.        , 0.        , 0.        , ..., 0.        , 0.00817439,
        0.01467269],
       [0.        , 0.        , 0.        , ..., 0.        , 0.02179837,
        0.03611738],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01362398,
        0.03047404],
       ...,
       [1.        , 0.        , 1.        , ..., 0.26319502, 0.01089918,
        0.18510158],
       [1.        , 0.        , 1.        , ..., 0.10532503, 0.03269755,
        0.13205418],
       [1.        , 0.        , 1.        , ..., 0.15786999, 0.01089918,
        0.09480813]])

In [21]:
# transform the train and test set, and add on the datetime and count variables
data = pd.concat([data[ 'count'].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(data[scaling_feature]), columns=scaling_feature)],
                    axis=1)

In [22]:
data.head()

,count,season,holiday,workingday,weather,temp,humidity,windspeed,casual,registered
0,2.772589,0.0,0.0,0.0,0.0,0.635197,0.81,0.0,0.008174,0.014673
1,3.688879,0.0,0.0,0.0,0.0,0.612955,0.80,0.0,0.021798,0.036117
2,3.465736,0.0,0.0,0.0,0.0,0.612955,0.80,0.0,0.013624,0.030474
3,2.564949,0.0,0.0,0.0,0.0,0.635197,0.75,0.0,0.008174,0.011287
4,0.000000,0.0,0.0,0.0,0.0,0.635197,0.75,0.0,0.000000,0.001129


In [23]:
data.to_csv('X_train.csv',index=False)